In [9]:
import torch
from bertviz import model_view
from transformers import BertTokenizer, BertForSequenceClassification

model_version = r"E:/PyCharmWorkspace/LegalJudgment/utils/model/torch_pretrained_bert_multi_label/tmp/self"
do_lower_case = True
model = BertForSequenceClassification.from_pretrained(model_version, num_labels=202, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)


In [10]:
sentence_a = "被告人周某在越野车内窃得黑色手机。"
# sentence_b = "马某被孙某打伤"
sentence_b = None

hide_delimiter_attn = False

inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
token_type_ids = inputs['token_type_ids']
input_ids = inputs['input_ids']

logits = model(input_ids, token_type_ids=token_type_ids)
attention = logits[-1]

input_id_list = input_ids[0].tolist()  # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)

if sentence_b:
    sentence_b_start = token_type_ids[0].tolist().index(1)
else:
    sentence_b_start = None
if hide_delimiter_attn:
    for i, t in enumerate(tokens):
        if t in ("[SEP]", "[CLS]"):
            for layer_attn in attention:
                layer_attn[0, :, i, :] = 0
                layer_attn[0, :, :, i] = 0

In [11]:
# squeezed = []
# for layer_attention in attention:
#     # num_heads x seq_len x seq_len
#     squeezed.append(layer_attention.squeeze(0))
# import torch
# attn = torch.stack(squeezed)

In [12]:
# model_view(attention, tokens, sentence_b_start)

In [13]:
import numpy as np
attention_matrix = list(attention)
attention_matrix = [att.detach().numpy() for att in attention_matrix]

In [14]:
# attention_matrix = np.sum(attention_matrix, axis=0)[0]
# attention_matrix = np.sum(attention_matrix, axis=0)
# 
# attention_matrix = attention_matrix / (12 * 12)

In [15]:
result2save = logits[0].detach().numpy()[0]
attention2save = attention_matrix

In [16]:
result_s = [str(r) for r in result2save]
result_s = "\n".join(result_s)

with open("./result.txt", 'w', newline='\n') as f:
    f.write(result_s)

# attention_s = [" ".join([str(tmp) for tmp in list(i)]) for i in attention2save]
# attention_s = "\n".join(attention_s)
# with open("./attention.txt", "w", newline='\n') as f:
#     f.write(attention_s)

In [17]:
attention_json = {}
for i, lay in enumerate(attention2save):
    attention_json[i] = {}
    for j, head in enumerate(lay[0]):
        head2save = []
        for vec in head:
            head2save.append([str(tmp) for tmp in vec])
        attention_json[i][j] = head2save
import json
with open("./attention.json", "w") as f:
    json.dump(attention_json, f)

